In [55]:
# делал на основе блокнота на сайте mistral (в ссылках общего чата),
# на том же сайте выбил апи ключи


# чтобы поднять базовый пайплайн с rag практически ничего менять не потребовалось
# кроме отдельных случаев, о которых далее в ячейках

In [1]:
pip install faiss-cpu==1.7.4 mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.5/262.5 kB 12.6 MB/s eta 0:00:00


In [ ]:
from mistralai import Mistral
import requests
import numpy as np
import faiss
import os
from getpass import getpass

api_key= getpass("...")
client = Mistral(api_key=api_key)

In [57]:
# тут был вопрос, какой учебник собственно взять, так как все теханные материалы
# по матану в основном выдадут кучу латех формул, к тому же вероятно обрезанных
# поэтому было решено взять что-то гуманитарное и более-менее удобоваримое, содержащее только буквы

# UPD на кафедре читали курсы по sql в каких=то местах содержащие данные чисто в текстовом виде, что я и использую

In [96]:
f = open('date.txt', 'r')
text = f.read()
f.close()

In [97]:
len(text)

32432

In [98]:
chunk_size = 512
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

In [99]:
len(chunks)

64

In [100]:
def get_text_embedding(input):
    embeddings_batch_response = client.embeddings.create(
          model="mistral-embed",
          inputs=input
      )
    return embeddings_batch_response.data[0].embedding

In [101]:
import time

# mistral free дает только один запрос в секунду

In [102]:
embeddings_array = []

for chunk in chunks[:140]:
  emb_text = get_text_embedding(chunk)
  embeddings_array.append(emb_text)
  time.sleep(2.5)


In [104]:
text_embeddings = np.array(embeddings_array)

In [105]:
text_embeddings.shape

(64, 1024)

In [106]:
d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

In [110]:
def answerTheQuestion(question : str, model="mistral-large-latest"):
  question_embeddings = np.array([get_text_embedding(question)])
  D, I = index.search(question_embeddings, k=2)
  retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
  prompt = f"""
            Далее дан контекст информации
            ---------------------
            {retrieved_chunk}
            ---------------------
            Ответь на запрос только исходя из контекста выше
            Запрос: {question}
            Ответ:
            """
  messages = [
        {
            "role": "user", "content": prompt
        }
    ]
  chat_response = client.chat.complete(
        model=model,
        messages=messages
    )

  print(chat_response.choices[0].message.content)


In [114]:
answerTheQuestion("Что описывает модель данных?")

Модель данных описывает некоторый набор родовых понятий и признаков, которыми должны обладать все конкретные СУБД и управляемые ими базы данных, основанные на этой модели.
